In [1]:
import asyncio

## gather

[Документация](https://docs.python.org/3/library/asyncio-task.html#asyncio.gather)

1. Оборачивает все переданные awaitable в Задачи
2. Ожидает завершения всех задач в порядке их передачи
3. Возвращает упорядоченный список результатов



In [10]:
async def async_func1():
    await asyncio.sleep(2)
    print('✅ async_func1 done')
    return 2

async def async_func2():
    await asyncio.sleep(1)
    print('✅ async_func2 done')
    return 1

future = asyncio.gather(async_func1(), async_func2())
print(f'📝 {future=}')

result = await future
print(f'📝 {result=}')

📝 future=<_GatheringFuture pending>
✅ async_func2 done
✅ async_func1 done
📝 result=[2, 1]


❗ Если задача отменена или завершена с ошибкой, то остальные задачи не исполняются и выбрасывается исключение.

☝️ Но `return_exceptions=True` позволяет собрать исключения в список с остальными результатами. 


In [25]:
async def async_func_with_exc(n=3):
    await asyncio.sleep(n)
    raise Exception('error')


result = await asyncio.gather(
    async_func1(),
    async_func2(),
    async_func_with_exc(),
    return_exceptions=True,
)

print(result)

✅ async_func2 done
✅ async_func1 done
[2, 1, Exception('error')]


## TaskGroup

[Документация](https://docs.python.org/3/library/asyncio-task.html#asyncio.TaskGroup)

Задачи живут внутри контекста.\
На момент выхода из tg все задачи уже будут завершены/отменены

In [ ]:
from random import sample

async def afunc(s: float) -> float:
    await asyncio.sleep(s)
    return s

async with asyncio.TaskGroup() as tg:
    tasks = {
        tg.create_task(afunc(s))
        for s in sample(range(1,5), k=3)
        }
    print('Exiting from tg...')

print('After tg')
for task in tasks:
    print(task)
    print(task.result())


Exiting from tg...
After tg
<Task finished name='Task-98' coro=<afunc() done, defined at /var/folders/x2/4pfkx0w56k53mfx0b3lq3b700000gn/T/ipykernel_9951/1038869071.py:3> result=1>
1
<Task finished name='Task-97' coro=<afunc() done, defined at /var/folders/x2/4pfkx0w56k53mfx0b3lq3b700000gn/T/ipykernel_9951/1038869071.py:3> result=3>
3
<Task finished name='Task-99' coro=<afunc() done, defined at /var/folders/x2/4pfkx0w56k53mfx0b3lq3b700000gn/T/ipykernel_9951/1038869071.py:3> result=2>
2


### Порядок результатов
Порядок результатов определяется порядком ожидания внутри группы

In [40]:
async with asyncio.TaskGroup() as tg:
    task1 = tg.create_task(afunc(0.4))
    task2 = tg.create_task(afunc(0.3))
    task3 = tg.create_task(afunc(0.2))

    r1 = await task1
    r2 = await task2
    r3 = await task3

print(r1, r2, r3)
print(task1.result(), task2.result(), task3.result())


    

0.4 0.3 0.2
0.4 0.3 0.2


### Ошибка в задаче
При ошибке одной задачи, остальные отменяются автоматически.

In [47]:
try:
    async with asyncio.TaskGroup() as tg:
        tasks = {
            tg.create_task(afunc(s))
            for s in [0.5, 0.2]
            }
        tasks.add(
            tg.create_task(async_func_with_exc(n=1))
        )
        print('🏁 Выход из TaskGroup()')
except Exception as e:
    print('⛔  Ошибочка:', e)

print('✅ TaskGroup() выполнена')
for task in tasks:
    print(task)

🏁 Выход из TaskGroup()
⛔  Ошибочка: unhandled errors in a TaskGroup (1 sub-exception)
✅ TaskGroup() выполнена
<Task finished name='Task-174' coro=<afunc() done, defined at /var/folders/x2/4pfkx0w56k53mfx0b3lq3b700000gn/T/ipykernel_9951/1038869071.py:3> result=0.2>
<Task finished name='Task-173' coro=<afunc() done, defined at /var/folders/x2/4pfkx0w56k53mfx0b3lq3b700000gn/T/ipykernel_9951/1038869071.py:3> result=0.5>
<Task finished name='Task-175' coro=<async_func_with_exc() done, defined at /var/folders/x2/4pfkx0w56k53mfx0b3lq3b700000gn/T/ipykernel_9951/1789693524.py:1> exception=Exception('error')>


### Отмена задачи
Отмена одной задачи отменяет все задачи

In [56]:
async def afunc_with_try():
    try:
        await asyncio.sleep(0.2)
    except asyncio.CancelledError:
        print('Canceling function...')
        raise

async with asyncio.TaskGroup() as tg:
    t1 = tg.create_task(afunc_with_try())
    t2 = tg.create_task(afunc(0.3))

    task1.cancel()
    try:
        await t1
        await t2
    except asyncio.CancelledError:
        print('Cancel TastGroup()')
print(t1.result(), t2.result())

None 0.3
